**기울기 소실(Gradient Vanishing)** : 인공 신경망 학습 중 역전파 과정에서 입력층으로 갈수록 기울기가 점차적으로 작아지는 현상
**기울기 폭주(Gradient Exploding)** : 반대로 기울기가 점차 커지더니 가중치들이 비정상적으로 큰 값이 되면서 결국 발산하는 현상


*기울기 소실 또는 기울기 폭주를 막는 방법*
## 1. ReLU와 ReLU의 변형들
* 시그모이드 함수를 사용하면 입력의 절대값이 클 경우 시그모이드 함수의 출력값이 0 또는 1에 수렴하면서 기울기가 0에 가까워져 역전파 과정에서 전파시킬 기울기가 점차 사라져 입력층 방향으로 갈수록 제대로 역전파가 되지 않는 기울기 소실 문제 발생
* 기울기 소실을 완화하는 가장 간단한 방법은 은닉층의 활성화 함수로 시그모이드나 하이퍼볼릭탄젠트 함수 대신에 **ReLU나 ReLU의 변형 함수와 같은 Leaky ReLU를 사용하는 것**


## 2. 그래디언트 클리핑(Gradient Clipping)
* 기울기 폭주를 막기 위해 임계값을 넘지 않도록 기울기 값을 자른다. 임계치만큼 크기 감소
* RNN은 BPTT에서 시점을 역행하면서 기울기를 구하는 데, 이때 기울기가 너무 커질 수 있기 때문에 RNN에서 유용

```from tensorflow.keras import optimizers```

```Adam = optimizers.Adam(lr=0.0001, clipnorm=1.)```


## 3. 가중치 초기화(Weight initialization)
같은 모델을 훈련시키더라도 가중치가 초기에 어떤 값을 가졌느냐에 따라서 모델의 훈련 결과가 달라지기도 한다. => 가중치 초기화만 적절히 해도 기울기 소실 문제 완화 가능

### 1) 세이비어 초기화(Xavier Initialization)
* 균등 분포(Uniform Distribution) 또는 정규 분포(Normal distribution)로 초기화 할 때 두 가지 경우로 나뉜다.
* 이전 층의 뉴런 개수와 다음 층의 뉴런 개수를 가지고 식을 세운다.
* 여러 층의 기울기 분산 사이에 균형을 맞춰서 특정 층이 너무 주목을 받거나 다른 층이 뒤쳐지는 것을 막는다.
* 세이비어 초기화는 시그모이드 함수나 하이퍼볼릭 탄젠트 함수와 같은 S자 형태인 활성화 함수와 함께 사용할 경우에는 좋은 성능을 보이지만, ReLU와 함께 사용할 경우에는 성능이 좋지 않다.

### 2) He 초기화(He initialization)
* 세이비어 초기화와 유사하게 정규 분포와 균등 분포 두 가지 경우로 나뉜다.
* 세이비어 초기화와 다르게 다음 층의 뉴런의 수를 반영하지 않는다.
* ReLU 계열 함수를 사용할 경우에 효율적


## 4. 배치 정규화(Batch Normalization)
ReLU 계열의 함수와 He 초기화를 사용하는 것만으로도 어느 정도 기울기 소실과 폭주를 완화시킬 수 있지만, 이 두 방법을 사용하더라도 훈련 중에 언제든 다시 발생할 수 있다.
* 배치 정규화는 인공 신경망의 각 층에 들어가는 입력을 평균과 분산으로 정규화하여 학습을 효율적으로 만든다.

### 1) 내부 공변량 변화(Internal Covariate Shift)
* 내부 공변량 변화란 학습 과정에서 층 별로 입력 데이터 분포가 달라지는 현상
* 이전 층들의 학습에 의해 이전 층의 가중치 값이 바뀌게 되면, 현재 층에 전달되는 입력 데이터의 분포가 현재 층이 학습했던 시점의 분포와 차이가 발생

### 2) 배치 정규화(Batch Normalization)
* 배치 정규화(Batch Normalization)는 표현 그대로 한 번에 들어오는 배치 단위로 정규화하는 것, 각 층에서 활성화 함수를 통과하기 전에 수행
* 입력에 대해 평균을 0으로 만들고, 정규화 -> 정규화 된 데이터에 대해서 스케일과 시프트를 수행 -> 두 개의 매개변수 γ와 β 중, γ는 스케일을 위해 사용하고, β는 시프트를 하는 것에 사용하며 다음 레이어에 일정한 범위의 값들만 전달되게 한다.
* 학습 시 배치 단위의 평균과 분산들을 차례대로 받아 이동 평균과 이동 분산을 저장해놓았다가 테스트 할 때는 해당 배치의 평균과 분산을 구하지 않고 구해놓았던 평균과 분산으로 정규화
* 배치 정규화를 사용하면 시그모이드 함수나 하이퍼볼릭탄젠트 함수를 사용하더라도 기울기 소실 문제가 크게 개선
* 가중치 초기화에 훨씬 덜 민감
* 훨씬 큰 학습률을 사용할 수 있어 학습 속도를 개선
* 미니 배치마다 평균과 표준편차를 계산하여 사용하므로 훈련 데이터에 일종의 잡음 주입의 부수 효과로 과적합을 방지하는 효과, 드롭아웃과 비슷한 효과(그래도 드롭 아웃과 함께 사용하는 것이 좋음)
* 배치 정규화는 모델을 복잡하게 하며, 추가 계산을 하는 것이므로 테스트 데이터에 대한 예측 시에 실행 시간이 느려지는 단점이 있다.

### 3) 배치 정규화의 한계
1. 미니 배치 크기에 의존적이다. => 너무 작은 배치 크기에서는 잘 작동하지 않음
2. RNN에 적용하기 어렵다. => RNN은 각 시점(time step)마다 다른 통계치를 가져 배치 정규화를 적용하는 것이 어렵다.


## 5. 층 정규화(Layer Normalization)
* 배치크기에 의존적이지 않으며, RNN에도 적용이 가능
* 동일한 특성(feature) 개수들을 가진 다수의 샘플인 미니 배치에서 특성별로 평균과 표준 편차를 구하는 것이 아닌 샘플별로 평균과 표준편차를 구한다.